In [1]:
!pip install transformers torch nltk rouge_score

In [1]:
import math
import random
import sys
import os
# from framework.utilities.models import inference    
from framework.utilities.metrics import calculate_bleu, calculate_rouge
import fastText
from contextlib import contextmanager
import time
import json

In [2]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
import pickle
import pandas as pd

with open('data/combined_outputs.pkl', 'rb') as file:
    combined_outputs = pickle.load(file)

df_combined_outputs = pd.DataFrame(combined_outputs)
df_combined_outputs.head()

,wmt14,cnn_dailymail,gsm8k
0,"{'input': 'Eine republikanische Strategie, um ...","{'input': '(CNN)Share, and your gift will be m...",{'input': 'Janet’s ducks lay 16 eggs per day. ...
1,{'input': 'Die Führungskräfte der Republikaner...,"{'input': '(CNN)On the 6th of April 1996, San ...",{'input': 'A robe takes 2 bolts of blue fiber ...
2,{'input': 'Allerdings hält das Brennan Center ...,{'input': '(CNN)French striker Bafetimbi Gomis...,{'input': 'Josh decides to try flipping a hous...
3,{'input': 'Die Rechtsanwälte der Republikaner ...,{'input': '(CNN)It was an act of frustration p...,{'input': 'James decides to run 3 sprints 3 ti...
4,{'input': 'Eins ist sicher: diese neuen Bestim...,{'input': '(CNN)A Pennsylvania community is pu...,"{'input': 'Every day, Wendi feeds each of her ..."


In [4]:
df_combined_outputs.size

3000

In [5]:
df_combined_outputs["wmt14"][0]

{'input': 'Eine republikanische Strategie, um der Wiederwahl von Obama entgegenzutreten',
 'label': 'A Republican strategy to counter the re-election of Obama',
 '7b': ["A Republican Strategy to Avoid Obama's Reelection"],
 'tiny': ["A Republican Strategy to Avoid Obama's Reelection"],
 '13b': ["A republican strategy to counter Obama's re-election"]}

In [6]:
def algorithm(T, c):
    l_predictor, s_predictor = None, None
    results = []
    
    for t in range(1, T+1):
        print(f"Index: {t}")
        t_i = df_combined_outputs["cnn_dailymail"][t]["input"]
        t_e = df_combined_outputs["cnn_dailymail"][t]["label"]

        p_t = min(1, c/math.sqrt(t))
        X_t = Bernoulli(p_t)
        
        if X_t == 1:
            t_c = getResults(t, t_i, t_e)
            l_predictor, s_predictor = sgdStep(t_c)
            l_predictor, s_predictor = checkpoint(l_predictor, s_predictor, t, p_t)

        output, t_acc = queryBest(t, t_i, t_e, l_predictor, s_predictor)
        print(t_acc)
        results.append({
            'iteration': t,
            'input': t_i,
            'best_output': output,
            'chosen_model_accuracy': t_acc
        })

    return results

In [7]:
def Bernoulli(p_t):
    return random.random() < p_t 

In [8]:
def queryBest(t, t_i, t_e, l_predictor, s_predictor):
    l_acc, s_acc = predict(t_i, l_predictor, s_predictor)
    if l_acc > s_acc: # use larger model, e.g. 7b llama
        output = df_combined_outputs["cnn_dailymail"][t]["13b"]
    else: # use smaller model for all other cases, e.g. 3b llama
        output =  df_combined_outputs["cnn_dailymail"][t]["tiny"]

    # return output, calculate_bleu(output[0], t_e)
    return output, calculate_rouge(output[0], t_e)

In [9]:
def sgdStep(t_c):
    with open("fasttext_large.txt", "w") as f:
        f.write(f"__label__{t_c['large_model_accuracy']} {t_c['input_text']}\n")
    with open("fasttext_small.txt", "w") as f:
        f.write(f"__label__{t_c['small_model_accuracy']} {t_c['input_text']}\n")

    if os.path.exists("large_predictor.bin"):
        large_model_predictor = fastText.train_supervised(input="fasttext_large.txt", epoch=1, lr=1.0, wordNgrams=2, inputModel="large_predictor.bin")
    else:
        large_model_predictor = fastText.train_supervised(input="fasttext_large.txt", epoch=1, lr=1.0, wordNgrams=2)

    if os.path.exists("small_predictor.bin"):
        small_model_predictor = fastText.train_supervised(input="fasttext_small.txt", epoch=1, lr=1.0, wordNgrams=2, inputModel="small_predictor.bin")
    else:
        small_model_predictor = fastText.train_supervised(input="fasttext_small.txt", epoch=1, lr=1.0, wordNgrams=2)

    return large_model_predictor, small_model_predictor

In [10]:
def predict(text, l_predictor, s_predictor):
    l_predicted_label = l_predictor.predict(text)[0][0]
    l_predicted_accuracy = int(round(float(l_predicted_label.replace('__label__', ''))))

    s_predicted_label = s_predictor.predict(text)[0][0]
    s_predicted_accuracy = int(round(float(s_predicted_label.replace('__label__', ''))))

    return l_predicted_accuracy, s_predicted_accuracy

In [11]:
def getResults(t, t_i, t_e):
    s_output = df_combined_outputs["cnn_dailymail"][t]["tiny"]
    l_output = df_combined_outputs["cnn_dailymail"][t]["13b"]
    
    # WMT14
    # s_acc = calculate_bleu(s_output[0], t_e)
    # l_acc = calculate_bleu(l_output[0], t_e)

    # CNN Dailymail
    s_acc = calculate_rouge(s_output[0], t_e)['rouge1']
    l_acc = calculate_rouge(l_output[0], t_e)['rouge1']

    return {
        'input_text': t_i,
        'large_model_accuracy': l_acc,
        'small_model_accuracy': s_acc
    }

In [12]:
def checkpoint(l_predictor, s_predictor, t, p_t):
    if p_t > 0.5:
        if t % int(math.sqrt(t)) == 0:
            print("Checkpoint")
            l_predictor, s_predictor = save_models(l_predictor, s_predictor)
    else:
        if t % max(1, int(1 / math.sqrt(t))) == 0:
            print("Checkpoint")
            l_predictor, s_predictor = save_models(l_predictor, s_predictor)

    return l_predictor, s_predictor

In [13]:
def save_models(l_predictor, s_predictor):
    if l_predictor:
        l_predictor.save_model("large_predictor.bin")
    if s_predictor:
        s_predictor.save_model("small_predictor.bin")

    return l_predictor, s_predictor

In [14]:
results = algorithm(999, 5)
print(results)

Index: 1


Read 0M words
Number of words:  693
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:  260826 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  693
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:  683256 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
{'rouge1': 0.17475728155339806, 'rouge2': 0.05940594059405941, 'rougeL': 0.1553398058252427}
Index: 2


Read 0M words
Number of words:  281
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:  287058 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  281
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:  331848 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
{'rouge1': 0.380952380952381, 'rouge2': 0.1359223300970874, 'rougeL': 0.22857142857142856}
Index: 3


...

Read 0M words
Number of words:  223
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:  260217 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  223
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:  135636 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
{'rouge1': 0.4680851063829787, 'rouge2': 0.26086956521739135, 'rougeL': 0.4042553191489362}
Index: 988
{'rouge1': 0.29411764705882354, 'rouge2': 0.06, 'rougeL': 0.21568627450980393}
Index: 989
{'rouge1': 0.4339622641509434, 'rouge2': 0.23076923076923078, 'rougeL': 0.2830188679245283}
Index: 990


Read 0M words
Number of words:  399
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:  425703 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  399
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:  434511 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [15]:
# with open('data/wmt14_results.json', 'w') as f:
#     json.dump(results, f, indent=4)

In [15]:
with open('data/cnndailymail_results.json', 'w') as f:
    json.dump(results, f, indent=4)

In [4]:
import json

with open('data/wmt14_results.json', 'r') as f:
    data = json.load(f)

for result in data[:10]:
    print(result['chosen_model_accuracy'])

0.05312123894060139
0.2982895720543635
0.2830956597218095
0.708587641194393
0.06775875292600428
0.20377924119043478
0.15209404090446677
0.3033668865762665
0.5131275135405763
0.5028248236576276


In [5]:
total = 0
num_results = len(data)

for result in data:
    total += result['chosen_model_accuracy']

total / num_results

0.18766023212954513

In [16]:
with open('data/cnndailymail_results.json', 'r') as f:
    data = json.load(f)

In [17]:
rouge1_total = 0
rouge2_total = 0
rougeL_total = 0

num_results = len(data)

for result in data:
    rouge1_total += result['chosen_model_accuracy']['rouge1']
    rouge2_total += result['chosen_model_accuracy']['rouge2']
    rougeL_total += result['chosen_model_accuracy']['rougeL']

rouge1_avg = rouge1_total / num_results
rouge2_avg = rouge2_total / num_results
rougeL_avg = rougeL_total / num_results

print(f"Average ROUGE-1: {rouge1_avg}")
print(f"Average ROUGE-2: {rouge2_avg}")
print(f"Average ROUGE-L: {rougeL_avg}")

Average ROUGE-1: 0.2937870737089562
Average ROUGE-2: 0.10789698165405123
Average ROUGE-L: 0.19841589354653624
